# Minimum working example for dual zonotopes
Whirlwind tour of the codebase with most of the functionality explained (for the naive case)

Table of contents:
1. Loading a net and setting up an example verification problem 
2. Computing preactivation bounds / {boxes, zonotopes, polytopes}
3. Playing with the naive dual object 
4. Playing with the decomposed dual object
5. Interacting with zonotopes/partitioning

In [ ]:
# Basic import block 
import sys
sys.path.append('..')
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt


import abstract_domains as ad 
from neural_nets import FFNet, PreactBounds, KWBounds
import train
import utilities
import dual_naive as dn
import dual_decompose as dd 
import pickle
import seaborn as sns 
sns.set()

valsum = lambda d: sum(d.values()) # handy little function
flatten = lambda lol: [subel for sublist in lol for subel in sublist]

torch.manual_seed(42)

# Part 1: Loading a net and setting up an example

In [ ]:
### Load and train a net
# Simple [784, 256, 128, 64, 10] PGD-trained MNIST network

make_net = lambda: FFNet(
                    nn.Sequential(nn.Linear(784, 256), 
                    nn.ReLU(), 
                    nn.Linear(256, 128), 
                    nn.ReLU(), 
                    nn.Linear(128, 64), 
                    nn.ReLU(), 
                    nn.Linear(64, 10)))


adv_net = make_net() # Make network

mnist_train = train.load_mnist_data('train', batch_size=128) # load datasets
mnist_val = train.load_mnist_data('val')


headless_atk = train.PGD(None, float('inf'), 0.1, 10, lb=0.0, ub=1.0) #setup attack params
advtrain_params = train.TrainParameters(mnist_train, mnist_val, 10, adv_attack=headless_atk) # setup train params


try: # Try to load the pickled network, otherwise train it
    adv_net = pickle.load(open('../pickles/adv_net.pkl', 'rb'))
except:
    train.training_loop(adv_net, advtrain_params)
    pickle.dump(adv_net, open('../pickles/adv_net.pkl', 'wb'))
    train.test_validation(adv_net, advtrain_params)

advtrain_params.adv_attack = train.PGD(adv_net, float('inf'), 0.1, 10, lb=0.0, ub=1.0)
print('Clean acc: %.2f; Robust acc: %.2f' % train.test_validation(adv_net, advtrain_params)[1:])

In [ ]:
'''
Now convert network to a binary classifier: (can only do binary classifier certification).
We'll do the following jointly:
- pick an MNIST example to certify 
- build the Hyperbox that defines the adversarial input region (what the adv can do)
- build a Binary classifier of <label> vs <label + 1>  (e.g., if the example is a 7, this is a 7vs8 classifier)
'''



def setup_ex(x, network, rad): # handy function that does the steps above^
    # Returns bin_net, input_domain 
    test_box = ad.Hyperbox.linf_box(x.view(-1), rad) 
    ypred = network(x.view(1, -1)).max(dim=1)[1].item()
    
    bin_net = network.binarize(ypred, ((ypred +1) % 10))
    return bin_net, test_box


RAD = 0.1
test_ex = next(iter(mnist_val))[0][20].view(-1) #Just pick an arbitrary example
bin_net, test_input = setup_ex(test_ex, adv_net, RAD)
#test_input = test_input.clamp(0.0, 1.0)
print(bin_net(test_ex.view(1, 28, 28)))

# Part 2: Computing Preactivation Bounds
Explaining the Abstract Domain framework I've rebuilt for this


In [ ]:
""" All this stuff is contained in abstract_domains.py for extensions of the base AbstractDomain class.
    
    Ultimately we want to compute PreactBounds object which has the intermediate bounds stored in a list.
    The API is simple, see below for boilerplate methods for computing preactivation bounds
"""

def get_hyperbox_prespec(net, test_input):
    # Hyperbox bounds (interval bounds)
    bounds = PreactBounds(net, test_input, ad.Hyperbox)
    bounds.compute() 
    return bounds 

def get_zonobox_prespec(net, test_input):
    # Computes zonotopes, but then converts to hyperboxes
    bounds = PreactBounds(net, test_input, ad.Zonotope)
    bounds.compute() 
    
    bounds.abstract_domain = ad.Hyperbox 
    bounds.bounds = [_.as_hyperbox() for _ in bounds.bounds]
    
    return bounds 

def get_zonotope_prespec(net, test_input):
    # Computes zonotopes properly
    bounds = PreactBounds(net, test_input, ad.Zonotope)
    bounds.compute() 
    return bounds 

def get_polybox_prespec(net, test_input):
    # Computes polytopes [Kolter-Wong thing]
    # (bounds.bounds is boxes, but we store the whole polytope too)
    bounds = PreactBounds(net, test_input, ad.Polytope)
    bounds.compute()
    return bounds


def get_kw_prespec(net, test_input):
    bounds = KW


In [ ]:
# E.g., what I'm saying about zonotopes vs polytopes:
zono_bounds = get_zonotope_prespec(bin_net, test_input)
#poly_bounds = get_polybox_prespec(bin_net, test_input)

In [ ]:
print("ZONO BOUNDS: [%.2f, %.2f]" % (zono_bounds.bounds[-1].lbs.item(), zono_bounds.bounds[-1].ubs.item()))
#print("POLY BOUNDS: [%.2f, %.2f]" % (poly_bounds.bounds[-1].lbs.item(), poly_bounds.bounds[-1].ubs.item()))

# Part 3: Actually doing the dual verification
Let's use the setup from the previous block where we want to lower bound the optimum of minimize `bin_net(x)` over all `x` in `test_input`


In [ ]:
# For comparison, let's look at what happens when we use box-based inner minimizations
# (but intermediate bounds computed from zonotopes)

zonobox_bounds = get_zonobox_prespec(bin_net, test_input)
zonobox_dual = dn.NaiveDual(bin_net, test_input, preact_domain=ad.Hyperbox, 
                            prespec_bounds=zonobox_bounds, choice='naive')

optim_obj = optim.Adam(zonobox_dual.parameters(), lr=1e-2)


In [ ]:
zonobox_out = zonobox_dual.dual_ascent(1000, verbose=25, optim_obj=optim_obj)

In [ ]:
# On the other hand, we can do the same with zonotopes (no hyperbox cast)
# - run dual ascent for 1k iterations, and then start computing partition stuff 
zono_dual = dn.NaiveDual(bin_net, test_input, preact_domain=ad.Zonotope, 
                         choice='naive')

optim_obj = optim.Adam(zono_dual.parameters(), lr=1e-2)
zono_out = zono_dual.dual_ascent(1000, verbose=25, optim_obj=optim_obj)

In [ ]:
# And we can examine the contribution of each subproblem to the total lagrangian 
zono_dual.lagrange_by_var(zono_dual.argmin())

In [ ]:
# Start partitioning by modifying the choice and partition kwargs attr 

print("Lagrange bounds using naive inner min: ", zono_dual.lagrangian(zono_dual.argmin()))


zono_dual.choice = 'partition' 
zono_dual.partition_kwargs = {'num_partitions': 8, 'partition_style': 'fixed'} 
# num partition is # of partitions per zonotope 
# partition_style 'fixed' saves partitions, whereas 'random' re-partitions every time
print("Lagrange bounds when you partition: ", zono_dual.lagrangian(zono_dual.argmin()))


In [ ]:
# For this choice of dual variables lambda_, we can get bounds on the MIP subproblems (no partitioning) ...
# (note that further optimization of lambda_ will change these bounds)
est_bounds = zono_dual.lagrange_bounds({'TimeLimit': 10})

In [ ]:
# X's are solved exactly, Z's are tuples with upper/lower bounds from MIP
est_bounds

# Part 4: Decomposition Objects
There's an improved lagrangian formulation using lagrangian splitting, but the same idea holds: you can switch box-based relu programming problems to zonotope-based ones. There's some theory that this provides Lagrangians that are no worse than previous bounds, but the main benefit is that iteration is quicker (but the formulation is slightly more tricky to reason about) 

This is contained in the `dual_decompose.DecompDual` class, and the API is basically the same.

In [ ]:
zono_decomp = dd.DecompDual(bin_net, test_input,  preact_domain=ad.Zonotope, 
                            choice='naive', zero_dual=True)

# The only extra kwarg here is zero_dual, which initializes the dual variables 
# from the KW2017 paper, giving a slightly better initial bound 

zero_dual_bound = zono_decomp.lagrangian(zono_decomp.argmin())
zono_decomp = dd.DecompDual(bin_net, test_input,  preact_domain=ad.Zonotope, 
                            choice='naive', zero_dual=False)
init_dual_bound = zono_decomp.lagrangian(zono_decomp.argmin())

print("Zero dual bound: ", zero_dual_bound)
print("Init dual bound: ", init_dual_bound)


In [ ]:
optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-2)
zono_out = zono_decomp.dual_ascent(500, verbose=25, optim_obj=optim_obj)

optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-3)
zono_out = zono_decomp.dual_ascent(500, verbose=25, optim_obj=optim_obj)

In [ ]:
zono_decomp.choice = 'partition' 
zono_decomp.partition_kwargs = {'partition_style': 'fixed', 
                                'num_partitions': 16}
optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-2)

zono_out = zono_decomp.dual_ascent(25, verbose=1, optim_obj=optim_obj)

In [ ]:
zono_decomp.lagrange_bounds({'TimeLimit': 10})

# 5. Interacting with Zonotopes/Partitioning
Finally, we can consider the various ways we can partition/merge zonotope partitions.
First I'll go over how to modify the partitioning of the dual objects, then how to do this for zonotopes in general

In [ ]:
# Consider an existing dual object with some partitions 
print(zono_dual.choice)
print(zono_dual.partition_kwargs.keys())

In [ ]:
# Examining the actual partitions: 
# It's a dict with keys pointing to each layer's zonotope
zono_dual.partition_kwargs['partitions'].keys()

In [ ]:
# And each layer is a list of 
print(type(zono_dual.partition_kwargs['partitions'][1]))

# Where each element is a tuple like (idxs_of_original, zonotope)
zono_dual.partition_kwargs['partitions'][1][0]

In [ ]:
# The only things you'd probably want to do with a dual object is to 
# 1. reset the partitions 
# 2. merge existing partitions together 
zono_dual.partition_kwargs['partitions'] = None  # resents the partitions 


zono_dual.argmin() # Will remake the partitions 
zono_dual.shrink_partitions(4) # now 4 partitions per zonotope

In [ ]:
##### And to examine the individual zonotopes #####
zono_ex = zono_dual.preact_bounds.bounds[1]
zono_ex

In [ ]:
# You can get the center, generator, element-wise lower and upper bounds like 
print(zono_ex.center.shape, zono_ex.generator.shape)
print(zono_ex.lbs, zono_ex.ubs)

In [ ]:
# To solve a vanilla linear program over the zonotope:
zono_ex.solve_lp(torch.ones_like(zono_ex.lbs), get_argmin=True)

In [ ]:
# To solve a relu program: min_z c1@z + c2@relu(z)... 
c1 = torch.ones_like(zono_ex.lbs)
c2 = -torch.ones_like(zono_ex.lbs)
zono_ex.solve_relu_mip(c1, c2, apx_params={'TimeLimit': 10}, verbose=True)

In [ ]:
# To create partitions:
parts = zono_ex.make_random_partitions(10)
parts

In [ ]:
# and to merge partitions back together 
half_parts_a = ad.Zonotope.merge_partitions(parts[::2])
half_parts_b = ad.Zonotope.merge_partitions(parts[1::2])
half_parts_a

In [ ]:
#... and that's all, I think